### Instalando bibliotecas

In [22]:
! pip install pandas pyarrow faker

1838.00s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Defaulting to user installation because normal site-packages is not writeable


### Importando pacotes

In [23]:
import pandas as pd
import argparse
import random
import json
import sys
import time
import uuid
from datetime import datetime, timedelta
from faker import Faker

In [24]:
fake = Faker(locale='pt_BR')

In [25]:
class PedidoDeCompra:
    def __init__(self, produto, valor_unitario, quantidade, data_criacao, uf, id_cliente):
        self.id_pedido = str(uuid.uuid4()) 
        self.produto = produto
        self.valor_unitario = valor_unitario
        self.quantidade = quantidade
        self.data_criacao = data_criacao.isoformat()
        self.uf = uf
        self.id_cliente = id_cliente

    def to_dict(self):
        return self.__dict__

In [26]:
produtos = {
    "TV": 2500,
    "GELADEIRA": 2000,
    "HOMETHEATER": 500,
    "COMPUTADOR": 700,
    "MONITOR": 600,
    "TABLET": 1100,
    "SOUNDBAR": 900,
    "CELULAR": 1000,
    "NOTEBOOK": 1500
}

print(produtos)

{'TV': 2500, 'GELADEIRA': 2000, 'HOMETHEATER': 500, 'COMPUTADOR': 700, 'MONITOR': 600, 'TABLET': 1100, 'SOUNDBAR': 900, 'CELULAR': 1000, 'NOTEBOOK': 1500}


In [27]:
def gerar_pedido_aleatorio(data=None):
    if data is None:
        data = datetime.datetime.now()
    produto, valor_unitario = random.choice(list(produtos.items()))
    quantidade = random.randint(1, 3)
    data_criacao = data.replace(hour=random.randint(0, 23), minute=random.randint(0, 59), second=random.randint(0, 59))
    uf = fake.state_abbr()
    id_cliente = random.randint(1, 14000)
    return PedidoDeCompra(produto, valor_unitario, quantidade, data_criacao, uf, id_cliente)

In [28]:
data_pedido = datetime.strptime("02/01/2023", "%d/%m/%Y")
pedido1 = gerar_pedido_aleatorio(data=data_pedido);
print(pedido1.to_dict())
pedido2 = gerar_pedido_aleatorio(data=data_pedido);
print(pedido2.to_dict())

{'id_pedido': '8a7939de-3601-422a-8c53-54c464e4dbba', 'produto': 'NOTEBOOK', 'valor_unitario': 1500, 'quantidade': 1, 'data_criacao': '2023-01-02T06:14:23', 'uf': 'RN', 'id_cliente': 10373}
{'id_pedido': 'e1ab3d8f-c59a-48fc-8f68-73c5ee450614', 'produto': 'SOUNDBAR', 'valor_unitario': 900, 'quantidade': 1, 'data_criacao': '2023-01-02T00:05:13', 'uf': 'TO', 'id_cliente': 2140}


In [42]:
def gerar_pedidos_por_dia(data_pedido, quantidade_por_dia):
    df = pd.DataFrame()
    df = pd.DataFrame(columns=['id_pedido', 'produto', 'valor_unitario', 'quantidade', 'data_criacao', 'uf', 'id_cliente'])

    for _ in range(quantidade_por_dia):
        pedido = gerar_pedido_aleatorio(data_pedido)
        linha = {
            'id_pedido': pedido.id_pedido,
            'produto': pedido.produto,
            'valor_unitario': pedido.valor_unitario,
            'quantidade': pedido.quantidade,
            'data_criacao': pedido.data_criacao,
            'uf': pedido.uf,
            'id_cliente': pedido.id_cliente}
        df = pd.concat([df, pd.DataFrame([linha])], ignore_index=True)
    
    return df

In [47]:
data_pedido = datetime.strptime("01/01/2024", "%d/%m/%Y")
meu_df = gerar_pedidos_por_dia(data_pedido, 1000)

In [48]:
meu_df.head()

,id_pedido,produto,valor_unitario,quantidade,data_criacao,uf,id_cliente
0,1abdafc7-f893-4d76-884b-c42f921628b9,MONITOR,600,2,2024-01-01T14:36:40,PB,8927
1,ea84403d-c9f6-42c4-9115-4fec5cbec4d9,NOTEBOOK,1500,2,2024-01-01T06:52:42,RN,7891
2,56154443-5f4e-4b44-bb54-80c3d61d8dec,CELULAR,1000,1,2024-01-01T04:53:16,SC,114
3,ca05fb58-2429-4a58-b6cc-57b06cf0cfb1,GELADEIRA,2000,3,2024-01-01T10:55:37,PI,4
4,141c8296-9711-48c5-bf47-007e480a60d9,SOUNDBAR,900,3,2024-01-01T04:41:14,BA,5744


In [49]:
meu_df.shape

(1000, 7)

In [50]:
def main():

    quantidade = 1000
    data_inicio_str = '01/01/2024'
    data_fim_str = '31/01/2024' 
    data_inicio = datetime.strptime(data_inicio_str, "%d/%m/%Y")
    data_fim = datetime.strptime(data_fim_str, "%d/%m/%Y")

    data_atual = data_inicio
    while data_atual <= data_fim:
        data_atual_str = data_atual.strftime("%Y-%m-%d")
        for _ in range(quantidade):
            df = gerar_pedidos_por_dia(data_atual, quantidade)
            df.to_csv("./pedidos-" + data_atual_str + ".csv.gz", sep=';', header=True, index=False, mode='w', encoding='UTF-8', quoting=None, quotechar='"', doublequote=True, compression='gzip', chunksize=10000)

        data_atual += timedelta(days=1)
        print("Gerados pedidos para o dia " + data_atual_str)
 
if __name__ == '__main__':
    main()

Gerados pedidos para o dia 2024-01-01
Gerados pedidos para o dia 2024-01-02
Gerados pedidos para o dia 2024-01-03
Gerados pedidos para o dia 2024-01-04
Gerados pedidos para o dia 2024-01-05
Gerados pedidos para o dia 2024-01-06
Gerados pedidos para o dia 2024-01-07
Gerados pedidos para o dia 2024-01-08
Gerados pedidos para o dia 2024-01-09
Gerados pedidos para o dia 2024-01-10
Gerados pedidos para o dia 2024-01-11
Gerados pedidos para o dia 2024-01-12
Gerados pedidos para o dia 2024-01-13
Gerados pedidos para o dia 2024-01-14
Gerados pedidos para o dia 2024-01-15
Gerados pedidos para o dia 2024-01-16
Gerados pedidos para o dia 2024-01-17
Gerados pedidos para o dia 2024-01-18
Gerados pedidos para o dia 2024-01-19
Gerados pedidos para o dia 2024-01-20
Gerados pedidos para o dia 2024-01-21
Gerados pedidos para o dia 2024-01-22
Gerados pedidos para o dia 2024-01-23
Gerados pedidos para o dia 2024-01-24
Gerados pedidos para o dia 2024-01-25
Gerados pedidos para o dia 2024-01-26
Gerados pedi